In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

# Création de la session Spark
print("Initialisation de SparkSession")
spark = SparkSession.builder \
    .appName("Kafka Stream Mastodon") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.postgresql:postgresql:42.3.1") \
    .getOrCreate()
print("Initialisation de SparkSession réussie")

# Schéma des données JSON reçues depuis Kafka
schema = StructType([
    StructField("user", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("content", StringType(), True),
    StructField("hashtags", ArrayType(StringType()), True),
    StructField("favourites_count", IntegerType(), True),
    StructField("reblogs_count", IntegerType(), True)
])

# Configuration du topic Kafka et des serveurs
kafka_topic = "message_kafka"
kafka_bootstrap_servers = "kafka:9093"  # ou "localhost:9092"

# Lecture du flux depuis Kafka
print("Lecture des données de Kafka")
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("kafka.request.timeout.ms", "120000") \
    .load()
print("Lecture des données de Kafka réussie")

# Décodage du flux JSON
df_decoded = df.selectExpr("CAST(value AS STRING) as json_string") \
    .select(from_json(col("json_string"), schema).alias("data")) \
    .select("data.*")

# Fonction pour afficher les 5 premiers enregistrements
def process_batch(batch_df, batch_id):
    # Affiche les 5 premiers enregistrements
    print("Affichage des 5 premiers enregistrements:")
    batch_df.limit(5).show(truncate=False)

    # Insérer dans la base de données PostgreSQL
    postgres_url = "jdbc:postgresql://localhost:5432/mastodon_data"
    db_properties = {
        "user": "user",
        "password": "password",
        "driver": "org.postgresql.Driver"
    }

    # Insérer dans la table "mastodon_data"
    batch_df.write.jdbc(url=postgres_url, table="mastodon", mode="append", properties=db_properties)

# Lecture du flux avec une logique de traitement par batch
query_console = df_decoded.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .start()

query_console.awaitTermination()


Initialisation de SparkSession
Initialisation de SparkSession réussie
Lecture des données de Kafka
Lecture des données de Kafka réussie
Affichage des 5 premiers enregistrements:
+---------+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+----------------+-------------+
|user     |timestamp                |content                                                                                                                                                                                             

StreamingQueryException: [STREAM_FAILED] Query [id = dc02c152-de4c-455e-849c-e36bd8e40d60, runId = f6ed8ec8-fcfb-4209-b665-0dddfe3c77f4] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_62351/3604854213.py", line 58, in process_batch
    batch_df.write.jdbc(url=postgres_url, table="mastodon", mode="append", properties=db_properties)
  File "/usr/local/spark/python/pyspark/sql/readwriter.py", line 1984, in jdbc
    self.mode(mode)._jwrite.jdbc(url, 